# Hydrogen Atom Orbitals

This notebook uses an implementation of the equation given for wavefunctions in spherical coordinates given [here](https://en.wikipedia.org/wiki/Hydrogen_atom#Wavefunction), repeated below. There may still be some problems, as all my checks do not match re

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/5420e93c057b9abf30224e175d585fd48be03ba9" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -3.005ex; width:65.154ex; height:7.509ex;" alt="{\displaystyle \psi _{n\ell m}(r,\theta ,\phi )={\sqrt {{\left({\frac {2}{na_{0}^{*}}}\right)}^{3}{\frac {(n-\ell -1)!}{2n(n+\ell )!}}}}e^{-\rho /2}\rho ^{\ell }L_{n-\ell -1}^{2\ell +1}(\rho )Y_{\ell }^{m}(\theta ,\phi )}">

In [ ]:
from scipy.special import genlaguerre
from scipy.special import sph_harm
import numpy as np
import itertools

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt

%matplotlib notebook

[SciPy General Laguerre](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.genlaguerre.html)

[SciPy Spherical Harmonics](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html)

In [ ]:
bohr_radius = 1
# Most function definitions just use a bohr radius of 1, so terms may not be included.

def wavefunction(n, l, m, r, theta, phi):
    rho = (2 * r) / n
    term1 = np.sqrt(((2/n)**3) * (np.math.factorial(n-l-1) / (2 * n * np.math.factorial(n+l)) ))
    term2 = np.exp(-rho/2) * np.power(rho, l)
    laguerre = genlaguerre(n-l-1, 2*l+1)(rho)
    harmonic = sph_harm(m, l, phi, theta)
    
    return term1*term2*harmonic*laguerre
    
def spherical_to_cartesian(r, theta, phi):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    
    return x, y, z

def wavefunction_100(r):
    return (1/np.sqrt(np.pi)) * np.exp(-r/bohr_radius) * (1/bohr_radius) ** (3/2)

def wavefunction_210(r, theta, phi):
    term1 = 0.25*(1/np.sqrt(2*np.pi))
    term2 = r*np.exp(-r/2)
    term3 = np.cos(theta)
    
    return term1*term2*term3

def wavefunction_321(r, theta, phi):
    term1 = 1/(81*np.sqrt(np.pi))
    term2 = r**2
    term3 = np.exp(-r/3)
    term4 = np.sin(theta)*np.cos(theta)
    term5 = np.exp(1j*phi)
    
    return term1*term2*term3*term4*term5

# Coordinate set up

In [ ]:
# Set up coordinates - check our combinations to make sure 
# we're getting all of space
r = [10]
theta = np.linspace(-np.pi, np.pi, 45)
phi = np.linspace(-2*np.pi, 2*np.pi, 45)

combinations = np.array(list(itertools.product([10], theta, phi)))

x, y, z = spherical_to_cartesian(combinations[:,0], combinations[:,1], combinations[:,2])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z)

In [ ]:
# More dense numbers for calculations
r_range = np.linspace(0, 15, 30)
theta_range = np.linspace(-np.pi, np.pi, 90)
phi_range = np.linspace(-2*np.pi, 2*np.pi, 90)

# Use itertools instead of for loops to get combinations
combinations = np.array(list(itertools.product(r_range, theta_range, phi_range)))

print(f'Number of points {len(combinations)}')

# Get coordinates as variables
r = combinations[:, 0]
theta = combinations[:, 1]
phi = combinations[:, 2]

# Get cartesian for plotting
x, y, z = spherical_to_cartesian(r, theta, phi)

# Sanity Checks

This next section has some sanity checks to see if the written function matches the equations given for particular orbitals in my textbook. "Quantum Chemistry & Spectroscopy, Second Edition, Engel"

In [ ]:
# Calculate values for 1,0,0 wavefunction and check

wf_100 = wavefunction(1, 0, 0, r, theta, phi)

wf_100_simplified = wavefunction_100(r)

# Check that these are the same
assert np.allclose(wf_100.real, wf_100_simplified)

In [ ]:
# Plot the two functions vs r to see if they match.
fig = plt.figure()
plt.plot(r, list(wf_100.real))
plt.plot(r, wf_100_simplified, 'o')

In [ ]:
# Calculate values for 2,1,0 wavefunction

wf_210 = wavefunction(2, 1, 0, r, theta, phi)

wf_210_simplified = wavefunction_210(r, theta, phi)

# Check that these equations are equal
assert np.allclose(wf_210, wf_210_simplified)

# Plot vs R 
fig = plt.figure()
plt.plot(r, list(wf_210.real), 'o')
plt.plot(r, wf_210_simplified, '*')

# Plot vs theta
fig = plt.figure()
plt.plot(theta, list(wf_210.real), 'o')
plt.plot(theta, wf_210_simplified, '*')

In [ ]:
# Calculate the 321 wavefunction

wf_321 = wavefunction(3, 2, 1, r, theta, phi)

wf_321_simplified = wavefunction_321(r, theta, phi)

# Not quite sure what's going on here - seems to be shifted
plt.figure()
plt.title("Difference in calculated and expected values.")
plt.plot((wf_321 - wf_321_simplified))

#assert np.allclose(wf_321, wf_321_simplified)

In [ ]:
# They look very similar
plt.figure()
plt.plot(r,wf_321, 'o')
plt.plot(r,wf_321_simplified, '*')

In [ ]:
# Let's try plotting some surfaces.

# Start with 3D plot of 1s orbital where values are greater than
# the 90th percentile of values.

to_plot_mask = wf_100 > np.percentile(wf_100.real, 90)

wf_100_x = x[to_plot_mask]
wf_100_y = y[to_plot_mask]
wf_100_z = z[to_plot_mask]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(wf_100_x, wf_100_y, wf_100_z)

In [ ]:
# Let's try plotting some surfaces.

# Start with 3D plot of 1s orbital where values are greater than
# the 90th percentile of values.

wf_210_probability = wf_210 ** 2

to_plot_mask = wf_210_probability > np.percentile(wf_210_probability.real, 90)

wf_100_x = x[to_plot_mask]
wf_100_y = y[to_plot_mask]
wf_100_z = z[to_plot_mask]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(wf_100_x, wf_100_y, wf_100_z)

In [ ]:
# Let's try plotting some surfaces.

# Start with 3D plot of 1s orbital where values are greater than
# the 90th percentile of values.

wf_321_probability = wf_321 ** 2

to_plot_mask = wf_321_probability > np.percentile(wf_321_probability.real, 90)

wf_100_x = x[to_plot_mask]
wf_100_y = y[to_plot_mask]
wf_100_z = z[to_plot_mask]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(wf_100_x, wf_100_y, wf_100_z)